<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/Copia_de_CODI_DEFINITIU_ANONIMITZACIO_SQL_FAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langdetect
! pip install deep_translator
! pip install faker


import sqlite3
import pandas as pd
from langdetect import detect
from deep_translator import GoogleTranslator
from faker import Faker
import re

def traducir_texto(texto):
    """Traduce las líneas que no están en español."""
    lineas = texto.split('\n')
    lineas_traducidas = []

    for linea in lineas:
        try:
            idioma = detect(linea)
            if idioma != 'es':
                linea = GoogleTranslator(source='auto', target='es').translate(linea)
        except:
            pass  # Si no se puede detectar el idioma, se deja la línea igual

        lineas_traducidas.append(linea)

    return '\n'.join(lineas_traducidas)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=732b917f70352f634df8ccb4d1c85471e91b3ea9a7daaadec63bc70c39691c93
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.6 MB/s eta 0:00:00


In [ ]:


import re
from faker import Faker
import spacy
!python -m spacy download es_core_news_sm
def anonymizar_texto(texto):
    """Anonimiza datos sensibles en el texto, incluyendo nombres detectados por NER y frases específicas."""
    fake_es = Faker('es_ES')


    # Mapa para almacenar placeholders y sus nombres falsos correspondientes
    placeholder_map = {}
    placeholder_counter = 0

    # Patrones para detectar frases específicas y su idioma correspondiente
    patterns = [(r'(Me llamo\s+)([^.,;]+)', 'es')]  # Español


    # Procesar cada patrón para reemplazar nombres con placeholders
    for pattern, lang in patterns:
        matches = list(re.finditer(pattern, texto, flags=re.IGNORECASE))
        for match in reversed(matches):  # Procesar en orden inverso para evitar cambios en los offsets
            start_name = match.start(2)
            end_name = match.end(2)
            original_name = match.group(2).strip()

            # Generar nombre falso según el idioma
            if lang == 'es':
                fake_name = fake_es.name()


            # Crear y almacenar placeholder
            placeholder = f'{{{{NAME_{placeholder_counter}}}}}'
            placeholder_map[placeholder] = fake_name
            placeholder_counter += 1

            # Reemplazar el nombre original con el placeholder
            texto = texto[:start_name] + placeholder + texto[end_name:]

    # Anonimizar números de teléfono y DNI
    texto = re.sub(r'\b\d{9}\b', lambda x: fake_es.phone_number(), texto)
    texto = re.sub(r'\b\d{8}[A-Za-z]\b', lambda x: fake_es.ssn(), texto)

    # Cargar modelo de NER en español
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(texto)
    replacements = []

    # Detectar y reemplazar nombres mediante NER
    for ent in doc.ents:
        if ent.label_ == "PER":
            start = ent.start_char
            end = ent.end_char
            fake_name = fake_es.name()
            replacements.append((start, end, fake_name))

    # Aplicar reemplazos de NER en orden inverso
    for start, end, fake_name in sorted(replacements, key=lambda x: -x[0]):
        texto = texto[:start] + fake_name + texto[end:]

    # Reemplazar placeholders con nombres falsos correspondientes
    for placeholder, fake_name in placeholder_map.items():
        texto = texto.replace(placeholder, fake_name)

    return texto

def procesar_texto(texto):
    """Traduce el texto al español y luego aplica la anonimización."""
    # Asumiendo que existe una función traducir_texto definida en otro lugar
    texto_traducido = traducir_texto(texto)
    texto_anonimizado = anonymizar_texto(texto_traducido)
    return texto_anonimizado

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 55.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Conectar a la base de datos y crear la tabla
with sqlite3.connect("mi_base_de_datos.db") as conn:
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS mi_tabla (
        id INTEGER PRIMARY KEY,
        body TEXT,
        secret TEXT,
        direction TEXT,
        createdAt TEXT,
        OpenchannelAccountId INTEGER,
        OpenchannelInteractionId INTEGER,
        UserId INTEGER,
        ContactId INTEGER,
        AttachmentId INTEGER,
        sentBy TEXT
    );
    """)
    conn.commit()

    # Cargar datos desde Excel a la base de datos
    df = pd.read_excel("/content/MOSTRA_1.xlsx")  # Reemplazar con el nombre real del archivo
    df.to_sql("mi_tabla", conn, if_exists="replace", index=False)

    # Extraer la columna body
    df_body = pd.read_sql("SELECT id, body, createdAt, direction, ContactId FROM mi_tabla", conn)

    # Aplicar procesamiento a la columna body
    df_body['body'] = df_body['body'].apply(procesar_texto)

    # Actualizar la base de datos con los valores procesados
    for index, row in df_body.iterrows():
        cursor.execute("UPDATE mi_tabla SET body = ? WHERE id = ?", (row['body'], row['id']))

    conn.commit()

print("Base de datos creada y procesamiento completado.")

Base de datos creada y procesamiento completado.


In [ ]:
print(df_body.head())

      id                                               body  \
0  77623                                               Hola   
1  77624  Bienvenido a * Chat de apoyo emocional para jó...   
2  77625                  Me llamo Nieves Montalbán Carnero   
3  77626                                              Hola!   
4  77627  Antes de empezar a chatear, es necesario que l...   

             createdAt direction  ContactId  
0  2023-01-01 00:10:19        in        466  
1  2023-01-01 00:10:19       out        466  
2  2023-01-01 00:10:37        in        466  
3  2023-01-01 00:11:12       out        466  
4  2023-01-01 00:11:33       out        466  
